In [ ]:
# !pip install mistralai langchain-mistralai
# !pip install numpy faiss-cpu fastapi pydantic langchain langchain-community --quiet

In [ ]:
import os
from pymongo import MongoClient
from langchain.prompts import PromptTemplate
from langchain_mistralai import ChatMistralAI

MISTRAL_SMALL = 'mistral-small-2402'
MISTRAL_LARGE = 'mistral-large-latest'

In [18]:
mongo_client = MongoClient(os.environ['MONGO_URI'])
mongo_db = mongo_client['hack_ai_thon']

small_model = ChatMistralAI(model=MISTRAL_SMALL, temperature=0)
large_model = ChatMistralAI(model=MISTRAL_LARGE, temperature=0)

employees_collection = 'Employees'

In [29]:
def get_department_data(department):
    return [ent for ent in mongo_db[department].find()]


def get_user_intent(query):
    with open("prompt_user_intent.txt") as finput:
        base_prompt = finput.read()

    prompt_template = PromptTemplate(
        input_variables=["query"],
        template=(
            base_prompt + "\nQuery: {query}"
        )
    )
    prompt = prompt_template.format(query=query)
    response = small_model.invoke(prompt)
    return response.content


def verify_access(user_id, departments):
    user = mongo_db[employees_collection].find_one(
        {
            'EmployeeID' : user_id
        }
    )
    return all([user.get(dep + "Access") for dep in departments])

In [22]:
verify_access("82f213a8-780b-402b-9962-8eaf63ef56d1", ["HR"])

False

In [30]:
get_user_intent("Give the recent sales report")

"['Sales']"

In [32]:
# !pip install flask

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
Using cached flask-3.1.0-py3-none-any.whl (102 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from pydantic import BaseModel
# from fastapi import FastAPI
from flask import Flask

# app = FastAPI()
app = Flask(__name__)

# Request model
class QueryRequest(BaseModel):
    user: object
    query: str

@app.post("/query")
def handle_query():
    user = request.user
    user_intent, departments = get_user_intent(request.query)
    if len(departments) == 0:
        # response = generic_department_response()
        return response
    
    if verify_access(user['id'], departments):
        # response = get_query_response(user_intent)
        response = ""
        return response
    
    return {"error": "Unauthorized user"}

app.run(debug=True, load_dotenv=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

f:\Hack-AI-Thon 3.0\venv\Lib\site-packages\IPython\core\interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [37]:
%tb

SystemExit: 1